In [ ]:
endpoint='https://hf-mirror.com'

: 

In [ ]:
# download pretrained checkpoints
from huggingface_hub import hf_hub_download
ar_model_path = hf_hub_download('jiaqili3/vallex', 'valle_ar_mls_encodec.bin', endpoint=endpoint)
nar_model_path = hf_hub_download('jiaqili3/vallex', 'valle_nar_mls_encodec.bin', endpoint=endpoint)



: 

In [ ]:
from models.tts.valle_gpt_simple.valle_inference import ValleInference
# change to device='cuda' to use CUDA GPU for fast inference
model = ValleInference(use_vocos=True, ar_path=ar_model_path, nar_path=nar_model_path, device='cpu')

: 

In [ ]:
# prepare inference data
import librosa
wav = librosa.load()

: 